# Data Preprocessing

In [1]:
# Import packages
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np

Github: https://github.com/anetey-abbey/infoviz.git <br>
Notebook: https://anetey-abbey.github.io/infoviz/docs/notebook.html

## Load data

Import the country dataset to get the codes of the countries in the WDI dataset in order to differentiate from groups of countries, e.g. Europe.

In [2]:
# this dataset contains the meta data of the WDI dataset
df_countrydata = pd.read_csv('WDICountry.csv', sep=',') 

Import the WDI dataset and selected relevant indicators.

In [3]:
df = pd.read_csv('WDIData.csv', sep=',') 
df = df.drop('Unnamed: 66', axis=1)

# filter on period 1990 - 2020
years = list(map(str, range(1990, 2021)))
df = df[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'] + years]

# filter to keep only relevant variables in dataset
selected_indicators = ['GDP growth (annual %)',
                       'Individuals using the Internet (% of population)',
                       'Foreign direct investment, net inflows (BoP, current US$)',
                       'GNI per capita, PPP (current international $)',
                       'School enrollment, secondary (% net)',
                       'Individuals using the Internet (% of population)',
                        "Educational attainment, at least Bachelor's or equivalent, population 25+, total (%) (cumulative)",
                       'Educational attainment, at least completed primary, population 25+ years, total (%) (cumulative)',
                       "Educational attainment, at least Master's or equivalent, population 25+, total (%) (cumulative)",
                       'Educational attainment, Doctoral or equivalent, population 25+, total (%) (cumulative)',
                       'Government expenditure on education, total (% of GDP)',
                       'Secondary education, duration (years)',
                       'Secondary education, pupils',
                       'Adjusted savings: education expenditure (% of GNI)',
                       'Adjusted savings: education expenditure (current US$)',
                       'Compulsory education, duration (years)',
                       'Secure Internet servers',
                       'Fixed telephone subscriptions',
                       'Access to electricity (% of population)',
                       'ICT goods exports (% of total goods exports)',
                       'Mobile cellular subscriptions',
                       'Mobile cellular subscriptions (per 100 people)',
                       'Educational attainment, at least completed upper secondary, population 25+, total (%) (cumulative)',
                       'Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative)',
                       'School enrollment, tertiary (% gross)',
                       'School enrollment, secondary (% gross)',
                       'School enrollment, primary (% gross)',
                       'Literacy rate, youth total (% of people ages 15-24)',
                       'Literacy rate, adult total (% of people ages 15 and above)', 
                       'ICT goods imports (% total goods imports)']
df = df[df['Indicator Name'].isin(selected_indicators)]

# Print the head
df.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1990,1991,1992,1993,1994,1995,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
3,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,29.089827,31.844384,31.794160,32.001027,33.871910,38.880173,40.261358,43.061877,44.270860,45.803485
36,Africa Eastern and Southern,AFE,Adjusted savings: education expenditure (% of ...,NY.ADJ.AEDU.GN.ZS,4.426717,4.882468,6.162041,6.071125,6.04343,6.990009,...,4.755062,4.878013,4.563364,4.510963,4.433471,4.355900,4.442274,4.692845,4.791304,4.710650


For effective data analysis we need the data in the wide format but with the variable names in the columns.

In [4]:
df_long = df.melt(id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], 
                  var_name='Year', value_name='Value')

df_wide = df_long.pivot_table(index=['Year', 'Country Name', 'Country Code'], 
                              columns='Indicator Name', values='Value', aggfunc='first')

df_wide = df_wide.reset_index()
df_wide['Year'] = df_wide['Year'].astype(int)

df_wide

Indicator Name,Year,Country Name,Country Code,Access to electricity (% of population),Adjusted savings: education expenditure (% of GNI),Adjusted savings: education expenditure (current US$),"Compulsory education, duration (years)","Educational attainment, Doctoral or equivalent, population 25+, total (%) (cumulative)","Educational attainment, at least Bachelor's or equivalent, population 25+, total (%) (cumulative)","Educational attainment, at least Master's or equivalent, population 25+, total (%) (cumulative)",...,"Literacy rate, youth total (% of people ages 15-24)",Mobile cellular subscriptions,Mobile cellular subscriptions (per 100 people),"School enrollment, primary (% gross)","School enrollment, secondary (% gross)","School enrollment, secondary (% net)","School enrollment, tertiary (% gross)","Secondary education, duration (years)","Secondary education, pupils",Secure Internet servers
0,1990,Afghanistan,AFG,NaN,1.953448,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,0.000000,29.041420,10.849960,NaN,2.211410,6.0,182340.0,NaN
1,1990,Africa Eastern and Southern,AFE,NaN,4.426717,NaN,NaN,NaN,NaN,NaN,...,NaN,7.880000e+03,0.002587,NaN,NaN,NaN,NaN,6.0,NaN,NaN
2,1990,Africa Western and Central,AFW,NaN,2.615379,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,0.000000,NaN,NaN,NaN,NaN,7.0,NaN,NaN
3,1990,Albania,ALB,100.000000,2.800000,5.674630e+07,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,0.000000,99.451752,90.081741,NaN,8.366360,8.0,475074.0,NaN
4,1990,Algeria,DZA,NaN,4.946261,2.957000e+09,NaN,NaN,NaN,NaN,...,NaN,4.700000e+02,0.001825,92.593437,59.414188,NaN,10.293140,6.0,2162469.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8206,2020,West Bank and Gaza,PSE,100.000000,NaN,NaN,10.0,0.26999,21.708691,1.94667,...,99.232353,4.274119e+06,83.783026,96.427193,91.027290,NaN,43.097672,8.0,806276.0,2573.0
8207,2020,World,WLD,90.521569,3.872891,NaN,10.0,NaN,NaN,NaN,...,91.883980,8.265682e+09,106.152775,102.407066,76.750740,NaN,40.244061,6.0,613156032.0,89189073.0
8208,2020,"Yemen, Rep.",YEM,73.757927,3.600000,NaN,9.0,NaN,NaN,NaN,...,NaN,1.517800e+07,50.888548,NaN,NaN,NaN,NaN,6.0,NaN,169.0
8209,2020,Zambia,ZMB,44.524475,3.563864,6.273000e+08,7.0,NaN,NaN,NaN,...,NaN,1.910421e+07,103.917835,NaN,NaN,NaN,NaN,5.0,NaN,745.0


In [5]:
# join the continent data to the dataset
df_wide = pd.merge(df_wide, df_countrydata[['Country Code', 'Region']], on='Country Code', how='left')
df_wide

,Year,Country Name,Country Code,Access to electricity (% of population),Adjusted savings: education expenditure (% of GNI),Adjusted savings: education expenditure (current US$),"Compulsory education, duration (years)","Educational attainment, Doctoral or equivalent, population 25+, total (%) (cumulative)","Educational attainment, at least Bachelor's or equivalent, population 25+, total (%) (cumulative)","Educational attainment, at least Master's or equivalent, population 25+, total (%) (cumulative)",...,Mobile cellular subscriptions,Mobile cellular subscriptions (per 100 people),"School enrollment, primary (% gross)","School enrollment, secondary (% gross)","School enrollment, secondary (% net)","School enrollment, tertiary (% gross)","Secondary education, duration (years)","Secondary education, pupils",Secure Internet servers,Region
0,1990,Afghanistan,AFG,NaN,1.953448,NaN,NaN,NaN,NaN,NaN,...,0.000000e+00,0.000000,29.041420,10.849960,NaN,2.211410,6.0,182340.0,NaN,South Asia
1,1990,Africa Eastern and Southern,AFE,NaN,4.426717,NaN,NaN,NaN,NaN,NaN,...,7.880000e+03,0.002587,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN
2,1990,Africa Western and Central,AFW,NaN,2.615379,NaN,NaN,NaN,NaN,NaN,...,0.000000e+00,0.000000,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN
3,1990,Albania,ALB,100.000000,2.800000,5.674630e+07,NaN,NaN,NaN,NaN,...,0.000000e+00,0.000000,99.451752,90.081741,NaN,8.366360,8.0,475074.0,NaN,Europe & Central Asia
4,1990,Algeria,DZA,NaN,4.946261,2.957000e+09,NaN,NaN,NaN,NaN,...,4.700000e+02,0.001825,92.593437,59.414188,NaN,10.293140,6.0,2162469.0,NaN,Middle East & North Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8206,2020,West Bank and Gaza,PSE,100.000000,NaN,NaN,10.0,0.26999,21.708691,1.94667,...,4.274119e+06,83.783026,96.427193,91.027290,NaN,43.097672,8.0,806276.0,2573.0,Middle East & North Africa
8207,2020,World,WLD,90.521569,3.872891,NaN,10.0,NaN,NaN,NaN,...,8.265682e+09,106.152775,102.407066,76.750740,NaN,40.244061,6.0,613156032.0,89189073.0,NaN
8208,2020,"Yemen, Rep.",YEM,73.757927,3.600000,NaN,9.0,NaN,NaN,NaN,...,1.517800e+07,50.888548,NaN,NaN,NaN,NaN,6.0,NaN,169.0,Middle East & North Africa
8209,2020,Zambia,ZMB,44.524475,3.563864,6.273000e+08,7.0,NaN,NaN,NaN,...,1.910421e+07,103.917835,NaN,NaN,NaN,NaN,5.0,NaN,745.0,Sub-Saharan Africa


In [6]:
# most of the time we need the data of all countries only thus we filter out all non-country data
df_countrydata = df_countrydata[df_countrydata['Income Group'].notna()]
country_codes = df_countrydata['Country Code'].unique()
df_all_countries = df_wide[df_wide['Country Code'].isin(country_codes)]

In [7]:
# Replace 'nan' with np.nan
df_all_countries.replace('nan', np.nan, inplace=True)

# Convert columns to numeric data type
df_all_countries['Individuals using the Internet (% of population)'] = pd.to_numeric(df_all_countries['Individuals using the Internet (% of population)'], errors='coerce')

# Replace missing values with linearly interpolated values
df_all_countries = df_all_countries.interpolate(method='linear')

/var/folders/v9/_3m9v4153zx32g3v3h3vvfp00000gn/T/ipykernel_40171/1657942527.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_countries.replace('nan', np.nan, inplace=True)
/var/folders/v9/_3m9v4153zx32g3v3h3vvfp00000gn/T/ipykernel_40171/1657942527.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_countries['Individuals using the Internet (% of population)'] = pd.to_numeric(df_all_countries['Individuals using the Internet (% of population)'], errors='coerce')


In [8]:
# Save as pickle file. This datatype preserves all relevant information of the dataframe.
df_wide.to_pickle("df_wide.pkl")
df_all_countries.to_pickle("df_all_countries.pkl")

In [9]:
# Replace 'nan' with np.nan
df_all_countries.replace('nan', np.nan, inplace=True)

# Convert columns to numeric data type
df_all_countries['School enrollment, tertiary (% gross)'] = pd.to_numeric(df_all_countries['School enrollment, tertiary (% gross)'], errors='coerce')
df_all_countries['School enrollment, secondary (% gross)'] = pd.to_numeric(df_all_countries['School enrollment, secondary (% gross)'], errors='coerce')

# Replace missing values with linearly interpolated values
df_all_countries_clean = df_all_countries.interpolate(method='linear')

In [10]:
# Save as pickle file. This datatype preserves all relevant information of the dataframe.
df_all_countries_clean.to_pickle("df_all_countries_clean.pkl")